## 일단 아무런 처리 안 한 데이터로 catboost 돌려서 과적합 되는지 확인해보기

In [7]:
import numpy as np
import pandas as pd
import random
import os
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import random
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint, uniform
from sklearn.preprocessing import LabelEncoder
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, f1_score, mean_squared_error
import numpy as np
import time
from itertools import product
from catboost import CatBoostClassifier, CatBoostRegressor

In [8]:
def seed_everything(seed):
    random.seed(seed) #파이썬 자체 모듈 random 모듈의 시드 고정
    os.environ['PYTHONHASHSEED'] = str(seed) 
    np.random.seed(seed)
    
seed_everything(37) # Seed 고정/

In [9]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [10]:
train_a = train[train.PRODUCT_CODE=='A_31']
train_to = train[(train.PRODUCT_CODE=='T_31')|(train.PRODUCT_CODE=='O_31')]
test_a = test[test.PRODUCT_CODE=='A_31']
test_to = test[(test.PRODUCT_CODE=='T_31')|(test.PRODUCT_CODE=='O_31')]

In [11]:
train_a.head()

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,TRAIN_000,1,0.533433,2022-06-13 5:14,T050304,A_31,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
1,TRAIN_001,2,0.541819,2022-06-13 5:22,T050307,A_31,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2,TRAIN_002,1,0.531267,2022-06-13 5:30,T050304,A_31,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
3,TRAIN_003,2,0.537325,2022-06-13 5:39,T050307,A_31,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
4,TRAIN_004,1,0.531590,2022-06-13 5:47,T050304,A_31,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN


## train&test 데이터 준비

## A

In [21]:
train_y = train_a.Y_Class

In [23]:
train_x = train_a.drop(columns = ['PRODUCT_ID', 'Y_Class', 'Y_Quality', 'TIMESTAMP', 'PRODUCT_CODE'])
train_x

,LINE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,T050304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
1,T050307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2,T050304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
3,T050307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
4,T050304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,T050304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,51.71,59.64,54.61,57.05,63.18,1.0,NaN,NaN,NaN,NaN
584,T010305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
585,T010306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
594,T050304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,49.47,53.07,50.89,55.10,66.49,1.0,NaN,NaN,NaN,NaN


In [15]:
a_index = test_a.index.tolist()

In [19]:
test_x = test_a
test_x = test_x.drop(columns = ['PRODUCT_ID', 'TIMESTAMP', 'PRODUCT_CODE'], axis = 1)
test_x.head()

,LINE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,T010305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,T010306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,T010306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,T010305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,T050304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,57.74,52.51,54.45,57.99,63.16,1.0,NaN,NaN,NaN,NaN


In [26]:
from catboost import cv, Pool, CatBoostClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 범주형 특성
categorical_features = ['LINE']

# Pool 생성
train_pool = Pool(data=train_x, label=train_y, cat_features=categorical_features)

# 교차검증 실행
cv_results = cv(
    params={
        'iterations': 1000,
        'learning_rate': 0.03,
        'depth': 3,
        'l2_leaf_reg' : 10,
        'bagging_temperature' : 10,
        'loss_function': 'MultiClass',  # 다중 분류를 위한 손실 함수,
        'verbose': False,
    },
    pool=train_pool,
    fold_count=5,  # 5-겹 교차검증
    shuffle=True,
    partition_random_seed=37,
    stratified=True,
    early_stopping_rounds=50
)

# 교차검증 결과 출력
cv_results_df = pd.DataFrame(cv_results)
print(cv_results_df.head())

# 마지막 반복의 평균 및 표준오차 확인
mean_logloss = cv_results_df['test-MultiClass-mean'].iloc[-1]
std_logloss = cv_results_df['test-MultiClass-std'].iloc[-1]

print(f"최종 평균 Logloss: {mean_logloss:.4f}")
print(f"최종 Logloss 표준오차: {std_logloss:.4f}")

Training on fold [0/5]

bestTest = 0.939979456
bestIteration = 450

Training on fold [1/5]

bestTest = 0.8791918007
bestIteration = 838

Training on fold [2/5]

bestTest = 1.006058004
bestIteration = 175

Training on fold [3/5]

bestTest = 1.022929911
bestIteration = 130

Training on fold [4/5]

bestTest = 0.9350239438
bestIteration = 551

   iterations  test-MultiClass-mean  test-MultiClass-std  \
0           0              1.096355             0.001036   
1           1              1.094101             0.001138   
2           2              1.091636             0.000830   
3           3              1.088770             0.000905   
4           4              1.086563             0.001308   

   train-MultiClass-mean  train-MultiClass-std  
0               1.095105              0.001343  
1               1.091758              0.001783  
2               1.088849              0.001512  
3               1.085129              0.002705  
4               1.082262              0.003308  
최종 

In [27]:
#데이터 분리
# X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, 
#                                                     test_size=0.2, 
#                                                     stratify=train_y, 
#                                                     random_state=37)
# 1. 훈련 데이터와 테스트 데이터 분리
X_train_full, X_test, y_train_full, y_test = train_test_split(
    train_x, train_y, test_size=0.2, stratify=train_y, random_state=37
)

# 2. 훈련 데이터 내에서 검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, stratify=y_train_full, random_state=37
)

# 최종 모델 학습 (교차검증 이후 전체 데이터로 학습)
final_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.03,
    depth=3,
    l2_leaf_reg = 10,
    bagging_temperature = 10,
    cat_features=categorical_features,
    early_stopping_rounds=50,
    verbose=50
)

final_model.fit(
    X_train,
    y_train,
    eval_set=(X_val, y_val),  # 검증 데이터 설정
    early_stopping_rounds=50,   # 50번 연속 개선되지 않으면 종료
    verbose=50
)

preds_a = final_model.predict(test_x)

0:	learn: 1.0942075	test: 1.0945194	best: 1.0945194 (0)	total: 22.3ms	remaining: 22.3s
50:	learn: 0.9793805	test: 1.0317388	best: 1.0317388 (50)	total: 645ms	remaining: 12s
100:	learn: 0.9101783	test: 1.0023266	best: 1.0023266 (100)	total: 1.3s	remaining: 11.5s
150:	learn: 0.8556060	test: 0.9886435	best: 0.9878616 (143)	total: 1.93s	remaining: 10.8s
200:	learn: 0.8136872	test: 0.9781400	best: 0.9781400 (200)	total: 2.67s	remaining: 10.6s
250:	learn: 0.7749357	test: 0.9730712	best: 0.9730712 (250)	total: 3.31s	remaining: 9.88s
300:	learn: 0.7368206	test: 0.9659294	best: 0.9656234 (298)	total: 4.03s	remaining: 9.37s
350:	learn: 0.7023211	test: 0.9683342	best: 0.9626921 (327)	total: 4.67s	remaining: 8.64s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9626921209
bestIteration = 327

Shrink model to first 328 iterations.


In [28]:
y_pred = final_model.predict(X_test)
y_pred_proba = final_model.predict_proba(X_test)

# 테스트 데이터 평가 지표
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average="weighted")
roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class="ovr")
logloss = log_loss(y_test, y_pred_proba)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")
print(f"LogLoss: {logloss:.4f}")

Accuracy: 0.5600
F1-Score: 0.4625
ROC-AUC: 0.7163
LogLoss: 0.9476


In [29]:
preds_a = pd.DataFrame(preds_a)
preds_a.index = a_index
preds_a

,0
3,1
4,1
5,1
6,1
7,1
...,...
284,0
285,1
286,1
292,1


In [30]:
preds_a.describe()

,0
count,67.000000
mean,0.805970
std,0.398437
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000
